In [ ]:
import itertools

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
from spacy import displacy
from spacy.pipeline import EntityRuler
from spacy.lang.es.examples import sentences
from spacy.lang.es.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import string

In [ ]:
dataset_test = pd.read_json('dataset_es_test.json', lines = True)
dataset_test.head()

In [ ]:
np.random #.seed(56)
index_random = np.random.randint(0,high = dataset_test.shape[0])
descripcion = dataset_test.iloc[index_random].review_title
print(index_random, descripcion)

In [ ]:
nlp = spacy.load("es_core_news_sm")

In [ ]:
doc = nlp (descripcion)

In [ ]:
for token in doc:
    print(token.text)

In [ ]:
stopwords_spacy = list(STOP_WORDS)
print (stopwords_spacy)
len(stopwords_spacy)

In [ ]:
for token in doc:
    if token.is_stop == False:
        print(token)

In [ ]:
for token in doc:
    print (token.text, token.lemma_)

In [ ]:
for token in doc:
    print (token.text, token.pos_)

In [ ]:
puntua = string.punctuation + '¿!¡? + " "'
puntua

In [ ]:
puntua = string.punctuation + '¿!¡? + " "'
puntua

In [ ]:
def text_data_cleaning(sentence):
    doc=nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != '-PRON-':
            temp= token.lemma_.strip()
        else:
            temp = token
        tokens.append(temp)
        
    clean_tokens = []
    for token in tokens:
        if token not in stopwords_spacy and token not in puntua:
            clean_tokens.append(token)
            
    return " ".join(clean_tokens)

In [ ]:
text_data_cleaning(descripcion)

In [ ]:
dataset_test.review_title.apply(text_data_cleaning)

In [ ]:
dataset_test.head(3)

In [ ]:
dataset_test['review_title']= dataset_test.review_title.apply(text_data_cleaning)

In [ ]:
dataset_test

In [ ]:
dataset_test.to_csv('dataset_test.csv', index = False)

In [ ]:
dataset_test